In [2]:
import os
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM

import librosa

2023-11-10 18:12:42.107536: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-10 18:12:42.170480: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 18:12:42.458320: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 18:12:42.458365: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 18:12:42.460604: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [3]:
# https://hyunlee103.tistory.com/36

# # 데이터 전처리 함수
# def preprocess_data(wav_file_path, labels):
#     # WAV 파일을 스펙트로그램으로 변환하고 필요한 전처리 수행
#     # ...

#     # 라벨링된 데이터를 이진 벡터와 시작 시간으로 분리
#     binary_vectors = labels[:, :8]
#     start_times = labels[:, 8]
#     return spectrogram_data, binary_vectors, start_times

max_pad_len = 174
def extract_feature(file_name):
    print('file name :', file_name)
    try:
        # res_type='kaiser_fast' : resampy 더 빠른 방법
        # 여부에 따라 값이 다르게 나오긴 함
        # resampy
        # 다차원 리샘플링을 지원하며 오디오 애플리케이션에 매우 적합합니다.
        # 장기간 신호(예: 고품질 샘플링 속도에서 몇 분)의 경우 resampy는 scipy.signal.resample 보다
        # 상당히 빠르며 오디오 품질에서는 인지할 수 있는 차이가 거의 없습니다.
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0,0), (0, pad_width)), mode='constant')
        print(mfccs.shape)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        print(e)
        return None
    
    return mfccs

In [19]:
# label-data mapping 해서 저장
inputs = []
outputs = []

# label class {HH: 0, RC: 1, ST: 0, ..., rhythm: 0}

# 드럼 녹음 data path
root_path = "../data/wavs/record/"

# [pattern, per_drum]
drum_folders = os.listdir(root_path)
for drum_folder in drum_folders:
    drum_folder_path = os.path.join(root_path, drum_folder)
    element_folder = os.listdir(drum_folder_path)
    # pattern : [1_HH, 2_RC, ...], per_drum : [P1, P2, ...]
    for element in element_folder:
            element_folder_path = os.path.join(drum_folder_path, element)
            rhythm_folder = os.listdir(element_folder_path)
            # 04, 08, 16
            for rhythm in rhythm_folder:
                rhythm_folder_path = os.path.join(element_folder_path, rhythm)
                files = os.listdir(rhythm_folder_path)
                wav_files = [os.path.join(rhythm_folder_path, file) for file in files if file.endswith('.m4a')]
                txt_files = [file for file in files if file.endswith('.txt')]
                if len(txt_files) > 0:
                    f = open(os.path.join(rhythm_folder_path, txt_files[0]), 'r')
                    # 라벨링 값 가져오기
                    # 01000000	000
                    # 01000000	100
                    while True:
                        line = f.readline()
                        if not line: break
                        print(line)
                    f.close()

                # outputs = txt_files
                # print(wav_files)

01000000	000

01000000	100

01000000	200

01000000	300

01000000	000

01000000	050

01000000	100

01000000	150

01000000	200

01000000	250

01000000	300

01000000	350

01000000	000

01000000	025

01000000	050

01000000	075

01000000	100

01000000	125

01000000	150

01000000	175

01000000	200

01000000	225

01000000	250

01000000	275

01000000	300

01000000	325

01000000	350

01000000	375



NotADirectoryError: [Errno 20] Not a directory: '../data/wavs/record/per_drum/1_HH/label.txt'

In [ ]:

# for  wav_list:

# wav_files = [os.path.join(root_path, file) for file in wav_list if file.endswith('.wav')]
# print(wav_list)

# # for CC, HH, RC... folder open
# #   for 04, 08, 16 folder open
# #       get wav from mp4 files
# #       get label from label.txt

# # data는 우리가 리브로사로 추출한 mfccs라는 특성이고
# # class_label은 그 음향의 종류를 나타낸다.

# features = []
# for wav_file in wav_files:
#     data = extract_feature(wav_file)
#     class_label = 0
#     if 'Overhead' in wav_file:
#         class_label = 1
#     elif 'Snare' in wav_file:
#         class_label = 2
#     elif 'Tom' in wav_file:
#         class_label = 3
#     elif 'Bass' in wav_file:
#         class_label = 4
#     else:
#         class_label = 0
#     features.append([data, class_label])

# # Convert into a Panda dataframe 
# featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

In [7]:
# 데이터 불러오기 및 전처리
root = "../data/wavs/test/"
test = root + "Overhead Sample 30.wav"



wav_file_path = "path_to_your_wav_file.wav"
labels = np.array([[0, 1, 0, 0, 0, 0, 0, 0, 0],  # 예시 라벨링 데이터
                   [0, 0, 0, 0, 1, 0, 0, 0, 100]])  # 예시 라벨링 데이터
input_data, binary_vectors, start_times = preprocess_data(wav_file_path, labels)

# 모델 입력의 형태 설정
input_shape = input_data.shape[1:]

# 이진 벡터 예측 레이어
binary_vector_output = Dense(8, activation='sigmoid', name='binary_vector')(x)

# 시작 시간 예측 레이어
start_time_output = Dense(1, name='start_time')(x)

# 모델 정의
model = Model(inputs=input_layer, outputs=[binary_vector_output, start_time_output])

# 모델 컴파일
model.compile(optimizer='adam', loss={'binary_vector': 'binary_crossentropy', 'start_time': 'mean_squared_error'})

# 모델 훈련
model.fit(input_data, {'binary_vector': binary_vectors, 'start_time': start_times}, epochs=100, batch_size=32)